In [4]:
#r nuget:Microsoft.ML
#r nuget:Microsoft.ML.TimeSeries

Installed package Microsoft.ML.TimeSeries version 1.4.0

Installed package Microsoft.ML version 1.4.0

In [23]:
using System;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.TimeSeries;
using XPlot.Plotly;

In [7]:
public class SalesRecord
{
    [LoadColumn(0)] public string Month;
    [LoadColumn(1)] public float Sales;
}

public class SalesPrediction
{
    //vector to hold alert,score,p-value values
    [VectorType(3)] public double[] Prediction { get; set; }
}


In [11]:
// filename for data set
private static string dataPath = Path.Combine(Environment.CurrentDirectory, "shampoo-sales.csv");

// create the machine learning context
var context = new MLContext();

// load the data file
Console.Write("Loading data...");
var dataView = context.Data.LoadFromTextFile<SalesRecord>(path: dataPath, hasHeader: true, separatorChar: ',');
Console.WriteLine("done");

Loading data...done


In [15]:
// get an array of data points
var sales = context.Data.CreateEnumerable<SalesRecord>(dataView, reuseRowObject: false).ToArray();
display(sales.Take(10));

index,Month,Sales
0,1-Jan,266
1,2-Jan,145.9
2,3-Jan,183.1
3,4-Jan,119.3
4,5-Jan,180.3
5,6-Jan,168.5
6,7-Jan,231.8
7,8-Jan,224.5
8,9-Jan,192.8
9,10-Jan,122.9


In [18]:
// plot the data
var n = sales.Count();
var chart = Chart.Plot(
    new Graph.Scattergl()
    {
        x = Enumerable.Range(1,n),
        y = (from i in Enumerable.Range(0,n) select sales[i].Sales),
        mode = "line"
    }
);
chart.WithXTitle("Month");
chart.WithYTitle("Sales");
chart.WithTitle("Sales data");
chart.Width = 600;
chart.Height = 600;
display(chart);

In [27]:
// build a training pipeline for detecting spikes
var pipeline = context.Transforms.DetectIidSpike(
    outputColumnName: nameof(SalesPrediction.Prediction), 
    inputColumnName: nameof(SalesRecord.Sales),
    confidence: 95, 
    pvalueHistoryLength: n / 4); // 25% of x-range

// train the model
Console.Write("Training model...");
var model = pipeline.Fit(dataView);
Console.WriteLine("done");

Training model...done


In [29]:
// predict spikes in the data
var predictions = model.Transform(dataView);
var predictionArray = context.Data.CreateEnumerable<SalesPrediction>(predictions, reuseRowObject: false).ToArray();

// find the spikes in the data
var spikes = (from i in Enumerable.Range(0, predictionArray.Count()) 
              where predictionArray[i].Prediction[0] == 1
              select (Day: i, Sales: sales[i].Sales));

In [54]:
// plot the spikes
var n = sales.Count();
var chart = Chart.Plot(
    new Graph.Scattergl[]
    {
        new Graph.Scattergl() {
            x = Enumerable.Range(1,n),
            y = (from i in Enumerable.Range(0,n) select sales[i].Sales),
            mode = "line"
        },
        new Graph.Scattergl() {
            x = (from s in spikes select s.Day+1),
            y = (from s in spikes select s.Sales),
            mode = "markers",
            marker = new Graph.Marker()
            {
                symbol = "square",
                size = 16
            }
        }
    }
);
chart.WithXTitle("Month");
chart.WithYTitle("Sales");
chart.WithTitle("Sales data");
chart.Width = 600;
chart.Height = 600;
chart.WithLegend(false);
display(chart);

In [56]:
// build a training pipeline for detecting change points
var pipeline2 = context.Transforms.DetectIidChangePoint(
    outputColumnName: nameof(SalesPrediction.Prediction), 
    inputColumnName: nameof(SalesRecord.Sales), 
    confidence: 95, 
    changeHistoryLength: n / 4); // 25% of x-range

// train the model
Console.Write("Training model...");
var model2 = pipeline2.Fit(dataView);
Console.WriteLine("done");

Training model...done


In [57]:
// get predictions
predictions = model2.Transform(dataView);
predictionArray = context.Data.CreateEnumerable<SalesPrediction>(predictions, reuseRowObject: false).ToArray();

// find the change points in the data
var changePoints = (from i in Enumerable.Range(0, predictionArray.Count()) 
                    where predictionArray[i].Prediction[0] == 1
                    select (Day: i, Sales: sales[i].Sales));

In [58]:
// plot the change points
var n = sales.Count();
var chart = Chart.Plot(
    new Graph.Scattergl[]
    {
        new Graph.Scattergl() {
            x = Enumerable.Range(1,n),
            y = (from i in Enumerable.Range(0,n) select sales[i].Sales),
            mode = "line"
        },
        new Graph.Scattergl() {
            x = (from c in changePoints select c.Day+1),
            y = (from c in changePoints select c.Sales),
            mode = "markers",
            marker = new Graph.Marker()
            {
                symbol = "square",
                size = 16
            }
        }
    }
);
chart.WithXTitle("Month");
chart.WithYTitle("Sales");
chart.WithTitle("Sales data");
chart.Width = 600;
chart.Height = 600;
chart.WithLegend(false);
display(chart);